In [1]:
import os
import xarray as xr
import numpy as np
from hydromt_sfincs import SfincsModel
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import scipy.io as sio
import sys
import os
import glob
import hydromt
import pyproj
import shapely
from hydromt import DataCatalog
import xarray as xr
import numpy as np
from os.path import join
import geopandas as gpd
import pandas as pd
from hydromt_sfincs import SfincsModel, utils
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors, patheffects
from string import ascii_lowercase as abcd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from matplotlib.ticker import FormatStrFormatter
import shapely

sys.path.append(r'C:\Users\lelise\Documents\GitHub\flood_model_carolinas\syntheticTCs_cmpdfld')
from src.utils import track_points_to_linestring

In [2]:
# PLOTTING MAPS
yml_base = r'Z:\Data-Expansion\users\lelise\data\data_catalog_BASE_Carolinas.yml'
base_root = r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\03_MODEL\sfincs_initcond_mod'
mod = SfincsModel(root=base_root, mode='r', data_libs=yml_base)
dem = mod.grid['dep']
region = mod.region

# Plotting details
wkt = mod.grid['dep'].raster.crs.to_wkt()
utm_zone = mod.grid['dep'].raster.crs.to_wkt().split("UTM zone ")[1][:3]
utm = ccrs.UTM(int(utm_zone[:2]), "S" in utm_zone)
font = {'family': 'Arial', 'size': 10}
mpl.rc('font', **font)
mpl.rcParams.update({'axes.titlesize': 10})
mpl.rcParams["figure.autolayout"] = True

In [3]:
# Load layers - run once because it takes a while...
coastal_wb = mod.data_catalog.get_geodataframe('carolinas_coastal_wb')
coastal_wb = coastal_wb.to_crs(mod.crs)
coastal_wb_clip = coastal_wb.clip(mod.region)

major_rivers = mod.data_catalog.get_geodataframe('carolinas_nhd_area_rivers')
major_rivers = major_rivers.to_crs(mod.crs)
major_rivers_clip = major_rivers.clip(mod.region)

nc_major_rivers = mod.data_catalog.get_geodataframe('carolinas_major_rivers')
nc_major_rivers = nc_major_rivers.to_crs(mod.crs)
nc_major_rivers_clip = nc_major_rivers.clip(mod.region)

# basins = mod.data_catalog.get_geodataframe(
#     r'Z:\users\lelise\geospatial\hydrography\nhd\NHD_H_North_Carolina_State_Shape\Shape\WBDHU10.shp',
#     bbox=mod.region.to_crs(4326).total_bounds)
# basins = basins.to_crs(mod.crs)
# zoom_newport = basins[basins['Name'] == 'Newport River'].buffer(3000).total_bounds

In [7]:
os.chdir(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter3_SyntheticTCs\04_RESULTS\SLR_analysis')

#zsmax_ds = xr.open_dataset(r'.\canesm_ssp585_SRL112cm\zsmax_canesm_slr.nc')
attr_ds = xr.open_dataset(r'.\canesm_ssp585_SRL112cm\hmin0.05\attribution_canesm_slr_0.05m.nc')

#zsmax_ds = xr.open_dataset(r'.\ncep_SRL112cm\zsmax_VmaxLf_44_115.nc')
#attr_ds = xr.open_dataset(r'.\ncep_SRL112cm\attribution_VmaxLf_44_115.nc')

# Mask out the cells that are considered water bodies
water_mask = xr.open_dataarray(r'..\waterbody_mask.nc')
water_mask = (water_mask == 0.0)

# Mask out water
#zsmax_ds = zsmax_ds.where(water_mask)
attr_ds = attr_ds.where(water_mask)

KeyboardInterrupt: 

In [5]:
# Calculate the compound flood extent
hmin = 0.05
compound_mask = attr_ds['zsmax_diff'] > hmin
cmpd_extent_ds = xr.where(compound_mask, x=1, y=0)

KeyError: "No variable named 'zsmax_diff'. Variables on the dataset include ['zsmax', 'y', 'spatial_ref', 'x', 'scenario', 'tc_id']"

In [ ]:
# Split the SLR and NO SLR IDs
tc_ids = cmpd_extent_ds.tc_id.values.tolist()
tc_ids_slr = [x for x in tc_ids if 'SLR' in x]
tc_ids_noslr = [x for x in tc_ids if 'SLR' not in x]

# Select the scenario runs and sum the frequency of being compound across all
cmpd_slr = cmpd_extent_ds.sel(tc_id=tc_ids_slr).sum(dim='tc_id')
cmpd_noslr = cmpd_extent_ds.sel(tc_id=tc_ids_noslr).sum(dim='tc_id')

max_num = len(tc_ids_slr)

# setup a mask for when both SLR and no SLR do not have compound flooding
data_mask = (cmpd_slr != 0) & (cmpd_noslr != 0)

# calculate the difference in the frequency of compound flooding across the events
diff = cmpd_slr - cmpd_noslr
diff2 = diff.where(data_mask)
#diff = xr.where(mask, np.nan, diff)

In [ ]:
# When compound 
# mask = (cmpd_slr > 0) & (cmpd_noslr == 0)
# data = cmpd_slr.where(mask)

# fig, ax = plt.subplots()

# data.plot(ax=ax, cmap='Reds')
# ax.set_title('')
# ax.set_aspect('equal')
# ax.set_axis_off()
# major_rivers_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=1)
# nc_major_rivers_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=1)
# coastal_wb_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=0.75)
# mod.region.plot(ax=ax, color='none', edgecolor='black', linewidth=0.75, zorder=3, alpha=1)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), subplot_kw={'projection': utm}, tight_layout=True)
cmap = 'coolwarm'
norm = mpl.colors.Normalize(vmin=-max_num, vmax=max_num)
cs = diff2.plot(ax=ax, 
               cmap=cmap, 
               norm=norm, 
               extend='neither', 
               shading='auto',
               add_colorbar=False, zorder=2, alpha=1)

ax.set_title('')
ax.set_aspect('equal')
ax.set_axis_off()
major_rivers_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=1)
nc_major_rivers_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=1)
coastal_wb_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=0.75)
mod.region.plot(ax=ax, color='none', edgecolor='black', linewidth=0.75, zorder=3, alpha=1)

ax.set_title(f'Frequency of Compound Flooding for Storms RP > 80-yr (hmin:{hmin}m)')
pos0 = ax.get_position()  # get the original position
cax = fig.add_axes([pos0.x1 + 0.02, pos0.y0 + pos0.height * 0.1, 0.05, pos0.y0 + pos0.height * 0.8])
cbar = fig.colorbar(cs,
                     cax=cax,
                     orientation='vertical',
                     ticks=[-max_num, 0, max_num],
                     label='Frequency'
                     )
cbar.ax.set_yticklabels(labels=['No SLR', 'Both', 'With SLR'])

plt.subplots_adjust(wspace=0.0, hspace=0)
plt.margins(x=0, y=0)
#plt.savefig(rf'.\ncep_cmpd_frequency_slr_hmin{hmin}.jpg', bbox_inches='tight', dpi=300)

In [ ]:
fignames = ['wilmington_zoom.png', 'new_bern_zoom.png', 'greenville_zoom.png', 'jacksonville_zoom.png', 'newport_river_zoom.png']
extents = [[730599.1620,3745648.4726,809931.1997,3832420.8957],
           [830375.7767,3870863.5373,878096.2839,3920289.7130],
           [817122.5776,3919050.7884,871781.0557,3966606.9186],
           [807146.7026,3821270.0954,863790.7176,3874636.9206],
           [867039.1461,3843681.4990,917857.7015,3881711.7466]
           ]
figsize = (5, 7)
for kk in range(len(fignames)):
    figname = fignames[kk]
    extent = extents[kk]
    polygon = shapely.geometry.box(*extent)
    plot_zoom=True
    if plot_zoom is True:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), subplot_kw={'projection': utm}, tight_layout=True)
        cmap = 'coolwarm'
        norm = mpl.colors.Normalize(vmin=-max_num, vmax=max_num)
        cs = diff.plot(ax=ax, 
                       cmap=cmap, 
                       norm=norm, 
                       extend='neither', 
                       shading='auto',
                       add_colorbar=False, zorder=2, alpha=1)
        
        ax.set_title('')
        ax.set_aspect('equal')
        ax.set_axis_off()
        major_rivers_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=1)
        nc_major_rivers_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=1)
        coastal_wb_clip.plot(ax=ax, color='slategrey', edgecolor='black', linewidth=0.25, zorder=0, alpha=0.75)
        mod.region.plot(ax=ax, color='none', edgecolor='black', linewidth=0.75, zorder=3, alpha=1)
        ax.plot(*polygon.exterior.xy, color='black', linewidth=1.5, zorder=3, alpha=1)

        minx, miny, maxx, maxy = extent
        ax.set_xlim(minx, maxx)
        ax.set_ylim(miny, maxy)
        
        ax.set_title(f'Frequency of Compound Flooding for Storms RP > 80-yr (hmin:{hmin}m)')
        pos0 = ax.get_position()  # get the original position
        cax = fig.add_axes([pos0.x1 + 0.02, pos0.y0 + pos0.height * 0.1, 0.05, pos0.y0 + pos0.height * 0.8])
        cbar = fig.colorbar(cs,
                             cax=cax,
                             orientation='vertical',
                             ticks=[-max_num, 0, max_num],
                             label='Frequency'
                             )
        cbar.ax.set_yticklabels(labels=['No SLR', 'Both', 'With SLR'])
        

        plt.subplots_adjust(wspace=0.0, hspace=0)
        plt.margins(x=0, y=0)
        plt.savefig(rf'.\{figname}_hmin{hmin}.jpg', bbox_inches='tight', dpi=300)